### Previous Code Version Sneaker Identifier

This was my original project code, but the tutorial that guided me through creating image classification was 6 years old, thus making this code outdated for this version of Python. In result, I used ChatGPT to help me transfer my code that would work for Python 3.5.3 to work for Python 3.10.6. This will explain the numerous pieces of code in identifier2.ipynb that are created using the assistance of ChatGPT

In [14]:
from PIL import Image
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt

TEST_DIR = '/Users/Lucas_Ying/Desktop/ATCS/Final Project_SneakerIdentifier/Sneaker-Identifier/data/TestingData'
TRAIN_DIR = '/Users/Lucas_Ying/Desktop/ATCS/Final Project_SneakerIdentifier/Sneaker-Identifier/data/TrainingData'

IMG_SIZE = 120
# learning rate
LR = 1e-3

MODEL_NAME = 'OFFWHITEvsYEEZY--{}-{}.model'.format(LR, '2conv-basic')

In [15]:

def label_img(img):
    # Images are formatted as: ADIDAS_1, NIKE_3 ...
    word_label = img.split('_')[0]
    if word_label == 'OFFWHITE': return [1,0] #one hot encoding
    elif word_label == 'YEEZY': return [0,1] #one hot encoding


In [16]:

def create_train_data():
    train_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        
        print(img)
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        if path == "/Users/Lucas_Ying/Desktop/ATCS/Final Project_SneakerIdentifier/Sneaker-Identifier/data/TrainingData/.DS_Store":
            break
        print("Loading image from path:", path)  # Debugging output to check the path
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))  # Read image in grayscale
        train_data.append([np.array(img), np.array(label)])
    shuffle(train_data)
    np.save('train_data.npy', train_data) #.npy extension = numpy file
    return train_data


In [17]:

def process_test_data():
    test_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('_')[1] #images are formatted 'OFFWHITE_2', 'YEEZY_56'..
        read = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(read, (IMG_SIZE,IMG_SIZE))
        
        test_data.append([np.array(img), img_num])
    shuffle(test_data)
    np.save('test_data.npy', test_data)
    return test_data


In [18]:
train_data = create_train_data()

  0%|          | 0/30 [00:00<?, ?it/s]

.DS_Store


In [19]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name = 'input')
convnet = conv_2d(convnet, 32, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation = 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = fully_connected(convnet, 1024, activation = 'relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 2, activation = 'softmax')
convnet = regression(convnet, optimizer = 'adam', learning_rate = LR, loss = 'categorical_crossentropy', name = 'targets')
model = tflearn.DNN(convnet, tensorboard_verbose=3)

In [21]:
# DATA SPLITTING
# Data splitting is typically done 80% train and 20% test
# of the 80% of our data in the train set
# we set aside a small percentage as a validation set
# this is used to do parameter tuning before evaluating
# on our test set

train = train_data[-90:] #Train Set
test = train_data[:-90] #Validation Set

In [22]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]

In [23]:
# model.fit({'input': X}, {'targets': Y}, n_epoch=100, validation_set=({'input': test_x}, {'targets': test_y}), snapshot_step=50, show_metric=True, run_id='OFFWHITE_YEEZY')


In [ ]:
test_data = process_test_data()
fig = plt.figure()

for num, data in enumerate(test_data[:10]):
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(3,4,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    model_out = model.predict([data])[0]
    print(model_out)
    if np.argmax(model_out) == 1:
        str_label = 'YEEZY'
    else:
        str_label = 'OFFWHITE'
    
    y.imshow(orig, cmap = 'gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()
